In [ ]:
%%bash
pip3 install kuzu

In [ ]:
import kuzu

db = kuzu.Database("./dblp")
conn = kuzu.Connection(db)

In [ ]:
import kuzu

db = kuzu.Database("./google")
conn = kuzu.Connection(db)

In [ ]:
conn.execute("CREATE NODE TABLE Dblp(id STRING, PRIMARY KEY (id))")
conn.execute("CREATE REL TABLE Connected(FROM Dblp TO Dblp)")

In [ ]:
conn.execute('COPY Dblp FROM "dblp-nodes.csv"')

In [ ]:
conn.execute('COPY Connected from "dblp.csv" (delim="\t")')

In [ ]:
conn.execute("CREATE NODE TABLE Google(id STRING, PRIMARY KEY (id))")
conn.execute("CREATE REL TABLE Connected(FROM Google TO Google)")

In [ ]:
conn.execute('COPY Google FROM "google-nodes.csv"')

In [ ]:
conn.execute('COPY Connected from "google.csv" (delim="\t")')

In [ ]:
import time

def run_query(query):
    start_time = time.time()
    results = conn.execute(query)
    while results.has_next():
        print(results.get_next())
    print("runtime: %s ms" % ((time.time() - start_time) * 1000))

In [ ]:
#DBLP

path03 = "MATCH (n1:Dblp)-[:CONNECTED]->(n2:Dblp)-[:CONNECTED]->(n3:Dblp)-[:CONNECTED]->(n4:Dblp)-[:CONNECTED]->(n5:Dblp) RETURN count(*)"
path04 = "MATCH (n1:Dblp)-[:CONNECTED]->(n2:Dblp)-[:CONNECTED]->(n3:Dblp)-[:CONNECTED]->(n4:Dblp)-[:CONNECTED]->(n5:Dblp)-[:CONNECTED]->(n6:Dblp) RETURN count(*)"
path05 = "MATCH (n1:Dblp)-[:CONNECTED]->(n2:Dblp)-[:CONNECTED]->(n3:Dblp)-[:CONNECTED]->(n4:Dblp)-[:CONNECTED]->(n5:Dblp)-[:CONNECTED]->(n6:Dblp)-[:CONNECTED]->(n7:Dblp) RETURN count(*)"
path06 = "MATCH (n1:Dblp)-[:CONNECTED]->(n2:Dblp)-[:CONNECTED]->(n3:Dblp)-[:CONNECTED]->(n4:Dblp)-[:CONNECTED]->(n5:Dblp)-[:CONNECTED]->(n6:Dblp)-[:CONNECTED]->(n7:Dblp)-[:CONNECTED]->(n8:Dblp) RETURN count(*)"
tree01 = "MATCH (n1:Dblp)-[:CONNECTED]->(n2:Dblp)-[:CONNECTED]->(n3:Dblp)-[:CONNECTED]->(n4:Dblp)-[:CONNECTED]->(n5:Dblp), (n4)-[:CONNECTED]->(n6:Dblp) RETURN count(*)"
tree02 = "MATCH (n1:Dblp)-[:CONNECTED]->(n2:Dblp)-[:CONNECTED]->(n3:Dblp)-[:CONNECTED]->(n4:Dblp)-[:CONNECTED]->(n5:Dblp), (n4)-[:CONNECTED]->(n6:Dblp), (n4)-[:CONNECTED]->(n7:Dblp) RETURN count(*)"

In [ ]:
#Google

path03 = "MATCH (n1:Google)-[:CONNECTED]->(n2:Google)-[:CONNECTED]->(n3:Google)-[:CONNECTED]->(n4:Google)-[:CONNECTED]->(n5:Google) RETURN count(*)"
path04 = "MATCH (n1:Google)-[:CONNECTED]->(n2:Google)-[:CONNECTED]->(n3:Google)-[:CONNECTED]->(n4:Google)-[:CONNECTED]->(n5:Google)-[:CONNECTED]->(n6:Google) RETURN count(*)"
path05 = "MATCH (n1:Google)-[:CONNECTED]->(n2:Google)-[:CONNECTED]->(n3:Google)-[:CONNECTED]->(n4:Google)-[:CONNECTED]->(n5:Google)-[:CONNECTED]->(n6:Google)-[:CONNECTED]->(n7:Google) RETURN count(*)"
path06 = "MATCH (n1:Dblp)-[:CONNECTED]->(n2:Dblp)-[:CONNECTED]->(n3:Dblp)-[:CONNECTED]->(n4:Dblp)-[:CONNECTED]->(n5:Dblp)-[:CONNECTED]->(n6:Dblp)-[:CONNECTED]->(n7:Dblp)-[:CONNECTED]->(n8:Dblp) RETURN count(*)"
tree01 = "MATCH (n1:Dblp)-[:CONNECTED]->(n2:Dblp)-[:CONNECTED]->(n3:Dblp)-[:CONNECTED]->(n4:Dblp)-[:CONNECTED]->(n5:Dblp), (n4)-[:CONNECTED]->(n6:Dblp) RETURN count(*)"
tree02 = "MATCH (n1:Dblp)-[:CONNECTED]->(n2:Dblp)-[:CONNECTED]->(n3:Dblp)-[:CONNECTED]->(n4:Dblp)-[:CONNECTED]->(n5:Dblp), (n4)-[:CONNECTED]->(n6:Dblp), (n4)-[:CONNECTED]->(n7:Dblp) RETURN count(*)"

In [ ]:
def benchmark(query):
    print(f'running query: {query}')
    for i in range(0,5):
        run_query(query)

benchmark(path05)

In [ ]:
run_query(tree02)

In [ ]:
nodeset = set()

#with open("snap/noheader/com-dblp.ungraph.txt") as file:
with open("snap/noheader/web-Google.txt") as file:
    for line in file:
        splits = line.split("\t")
        nodeset.add(splits[0])
        nodeset.add(splits[1].replace("\n", ""))

str = "\n".join(nodeset)

with open("google-nodes.csv", "w") as f:
    f.write(str)